# **_Data Loading Script_**

## **_Load the customer dimension data_**

In [136]:
Use TargetMart2
GO
INSERT INTO dim.Customers(CustomerID, Customer, City, Country, LoadDate, SourceCountry)
	SELECT nwc.CustomerID
		  ,nwc.CompanyName 
		  ,nwc.City
		  ,nwc.Country
		  ,CAST(getdate() as DATE)
		  ,concat('Country = ', nwc.Country) 
	FROM NORHTWIND.[dbo].[Customers] nwc
	WHERE nwc.CustomerID not in (SELECT CustomerID FROM dim.Customers)

;
GO

Commands completed successfully.

(0 rows affected)

Total execution time: 00:00:00.010

In [4]:
Select Top 10 * From dim.Customers;


(10 rows affected)

Total execution time: 00:00:00.012

pkCustomer,CustomerID,Customer,City,Country,LoadDate,SourceCountry
1000,ALFKI,Alfreds Futterkiste,Berlin,Germany,2023-11-30,Country = Germany
1001,ANATR,Ana Trujillo Emparedados y helados,México D.F.,Mexico,2023-11-30,Country = Mexico
1002,ANTON,Antonio Moreno Taquería,México D.F.,Mexico,2023-11-30,Country = Mexico
1003,AROUT,Around the Horn,London,UK,2023-11-30,Country = UK
1004,BERGS,Berglunds snabbköp,Luleå,Sweden,2023-11-30,Country = Sweden
1005,BLAUS,Blauer See Delikatessen,Mannheim,Germany,2023-11-30,Country = Germany
1006,BLONP,Blondesddsl père et fils,Strasbourg,France,2023-11-30,Country = France
1007,BOLID,Bólido Comidas preparadas,Madrid,Spain,2023-11-30,Country = Spain
1008,BONAP,Bon app',Marseille,France,2023-11-30,Country = France
1009,BOTTM,Bottom-Dollar Markets,Tsawassen,Canada,2023-11-30,Country = Canada


**_Simplicity: This script is used in the context of a data warehouse (TargetMart2) to load data into the dim.Customers dimension table. It inserts records from the source table \[NORTHWIND\].\[dbo\].\[Customers\] into the dimension table, including relevant customer details such as CustomerID, CompanyName, City, and Country._**

## _**Load the Calendar dimension table**_

In [138]:
Use TargetMart2
Go
IF (SELECT count(*) FROM dim.Calendar) = 0
BEGIN
-- Declare variables
DECLARE @StartDate DATE = '2020-01-01'
DECLARE @EndDate DATE = DATEADD(year, 0, GETDATE())
DECLARE @Date DATE = @StartDate
DECLARE @DayID INT = (datepart(year, @StartDate)-1900)*1000 + datepart(dy, @StartDate)
;


-- Populate the Calendar table
WHILE @Date <= @EndDate
	BEGIN
		INSERT INTO dim.Calendar (pkCalendar, DateValue, Year, Quarter, Qtr, Month,  MonthName, MonthShort, Week, Day, DayName, DayShort, IsWeekday, Weekday)
		VALUES (
			@DayID,
			@Date,
			YEAR(@Date),
			DATEPART(QUARTER, @Date),
			CASE WHEN DATEPART(QUARTER, @Date) IN (1) THEN '1st'
				 WHEN DATEPART(QUARTER, @Date) IN (2) THEN '2nd'
				 WHEN DATEPART(QUARTER, @Date) IN (3) THEN '3rd'
				 WHEN DATEPART(QUARTER, @Date) IN (4) THEN '4th'
				 ELSE '5th'
				 END,
			MONTH(@Date),
			DATENAME(MONTH, @Date),
			LEFT(DATENAME(MONTH, @Date),3),
			DATEPART(WEEK, @Date),
			DAY(@Date),
			DATENAME(WEEKDAY, @Date),
			LEFT(DATENAME(WEEKDAY, @Date),3),
			CASE WHEN DATEPART(WEEKDAY, @Date) IN (1, 7) THEN 0 ELSE 1 END, -- Set IsWeekday to 0 for Saturday (1) and Sunday (7), and 1 for weekdays
			CASE WHEN DATEPART(WEEKDAY, @Date) IN (1, 7) THEN 'No' ELSE 'Yes' END
	 )

		-- Increment the date and day ID
		SET @Date = DATEADD(DAY, 1, @Date)
		SET @DayID = @DayID + 1
	END
END
GO
;

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.005

## _**Load the Shipper dimension table**_

In [139]:
INSERT INTO dim.Shipper(pkShipId, Shipper, Phone)
SELECT s_sh.ShipperID
      ,s_sh.CompanyName
      ,s_sh.Phone
FROM NORHTWIND.dbo.Shippers s_sh
WHERE s_sh.ShipperID not in (SELECT pkShipId FROM dim.Shipper)
;
GO


(0 rows affected)

Total execution time: 00:00:00.010

In [5]:
Select  * from dim.Shipper ;

(3 rows affected)

Total execution time: 00:00:00.010

pkShipId,Shipper,Phone
1,Speedy Express,(503) 555-9831
2,United Package,(503) 555-3199
3,Federal Shipping,(503) 555-9931


## _**Load the Product dimension table**_

In [141]:
INSERT INTO dim.Products(pkProdId, ProductId, Product, ProductCategory, CategoryDesc, UnitPrice, UnitsInStock, UnitsOnOrder, ReorderLevel, Discontinued, ReorderFlag)
SELECT prod.ProductID + 10000 as 'pkProdId'
      , prod.ProductID as 'ProductId'
      , prod.ProductName as 'Product'
	  , cat.CategoryName as 'ProductCategory'
	  , cat.[Description] as 'CategoryDesc'
	  , prod.UnitPrice
	  , prod.UnitsInStock
	  , prod.UnitsOnOrder
	  , prod.ReorderLevel
	  , prod.Discontinued
	  , CASE 
			WHEN prod.ReorderLevel >= (prod.UnitsInStock + prod.UnitsOnOrder)
			     AND prod.Discontinued = 0
			 THEN 'Yes'
	      ELSE 'No'
	    END as 'ReorderFlag'
FROM NORHTWIND.dbo.Products prod
	INNER JOIN NORHTWIND.dbo.Categories cat
	ON prod.CategoryID = cat.CategoryID
WHERE prod.ProductID not in (SELECT ProductId FROM dim.Products)
;


(0 rows affected)

Total execution time: 00:00:00.016

In [6]:
Select top 10 * from dim.Products;

(10 rows affected)

Total execution time: 00:00:00.035

pkProdId,ProductId,Product,ProductCategory,CategoryDesc,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued,ReorderFlag
10001,1,Chai,Beverages,"Soft drinks, coffees, teas, beers, and ales",18.00,39,0,10,0,No
10002,2,Chang,Beverages,"Soft drinks, coffees, teas, beers, and ales",19.00,17,40,25,0,No
10003,3,Aniseed Syrup,Condiments,"Sweet and savory sauces, relishes, spreads, and seasonings",10.00,13,70,25,0,No
10004,4,Chef Anton's Cajun Seasoning,Condiments,"Sweet and savory sauces, relishes, spreads, and seasonings",22.00,53,0,0,0,No
10005,5,Chef Anton's Gumbo Mix,Condiments,"Sweet and savory sauces, relishes, spreads, and seasonings",21.35,0,0,0,1,No
10006,6,Grandma's Boysenberry Spread,Condiments,"Sweet and savory sauces, relishes, spreads, and seasonings",25.00,120,0,25,0,No
10007,7,Uncle Bob's Organic Dried Pears,Produce,Dried fruit and bean curd,30.00,15,0,10,0,No
10008,8,Northwoods Cranberry Sauce,Condiments,"Sweet and savory sauces, relishes, spreads, and seasonings",40.00,6,0,0,0,No
10009,9,Mishi Kobe Niku,Meat/Poultry,Prepared meats,97.00,29,0,0,1,No
10010,10,Ikura,Seafood,Seaweed and fish,31.00,31,0,0,0,No


### _**Simplicity : This script inserts data into the dim.Products dimension table in the TargetMart2 data warehouse. It transforms and selects information from the source tables \[NORTHWIND\].\[dbo\].\[Products\] and \[NORTHWIND\].\[dbo\].\[Categories\], calculating a derived primary key (pkProdId) and a ReorderFlag based on specific conditions. The insert operation ensures only new products, identified by ProductId, are added to the dimension table, maintaining data integrity.**_

\*\* **The ReorderFlag column is derived using a CASE statement. It evaluates conditions based on the ReorderLevel, UnitsInStock, UnitsOnOrder, and Discontinued columns to determine if the product needs to be reordered. If the conditions are met, it sets the flag to 'Yes', otherwise 'No'.**

## _**Load the Employees dimension table**_

In [143]:
INSERT INTO dim.Employees(EmployeeID, Employee, Title, BirthDate, HireDate, City, Country, ReportsTo)
SELECT sEmp.EmployeeID
	  ,concat( sEmp.TitleOfCourtesy, ' ', sEmp.FirstName, ', ', sEmp.LastName)
      ,sEmp.Title
      ,sEmp.BirthDate
      ,sEmp.HireDate
      ,sEmp.City
      ,sEmp.Country
      ,sEmp.ReportsTo
FROM NORHTWIND.dbo.Employees sEmp
WHERE sEmp.EmployeeID not in (SELECT EmployeeID FROM dim.Employees)

(0 rows affected)

Total execution time: 00:00:00.006

In [144]:
Select Top 10 * from dim.Employees;

(9 rows affected)

Total execution time: 00:00:00.007

EmployeeID,Employee,Title,BirthDate,HireDate,City,Country,ReportsTo
1,"Ms. Nancy, Davolio",Sales Representative,1973-12-08 00:00:00.000,2017-05-01 00:00:00.000,Seattle,USA,2
2,"Dr. Andrew, Fuller","Vice President, Sales",1977-02-19 00:00:00.000,2017-08-14 00:00:00.000,Tacoma,USA,NULL
3,"Ms. Janet, Leverling",Sales Representative,1988-08-30 00:00:00.000,2017-04-01 00:00:00.000,Kirkland,USA,2
4,"Mrs. Margaret, Peacock",Sales Representative,1962-09-19 00:00:00.000,2018-05-03 00:00:00.000,Redmond,USA,2
5,"Mr. Steven, Buchanan",Sales Manager,1980-03-04 00:00:00.000,2018-10-17 00:00:00.000,London,UK,2
6,"Mr. Michael, Suyama",Sales Representative,1988-07-02 00:00:00.000,2018-10-17 00:00:00.000,London,UK,5
7,"Mr. Robert, King",Sales Representative,1985-05-29 00:00:00.000,2019-01-02 00:00:00.000,London,UK,5
8,"Ms. Laura, Callahan",Inside Sales Coordinator,1983-01-09 00:00:00.000,2019-03-05 00:00:00.000,Seattle,USA,2
9,"Ms. Anne, Dodsworth",Sales Representative,1991-01-27 00:00:00.000,2019-11-15 00:00:00.000,London,UK,5


## **_Fact TableLoaders_**

## **_Order Perf Fact table_**

In [145]:
Begin
TRUNCATE TABLE TargetMart2.f.OrderPerf
INSERT INTO TargetMart2.f.OrderPerf(OrderID, fkCalendar, fkCustomer, fkShipper, fkEmployee, Freight, DaysToShip, DaysTilRequired, OrderCount)
SELECT sO.OrderID
	  ,tC.pkCalendar as fkCalendar
	  ,tCus.pkCustomer as fkCustomer
	  ,sO.ShipVia as fkShipper
	  ,sO.EmployeeID as fkEmployee
	  ,sO.Freight
	  ,DATEDIFF ( day , sO.OrderDate , sO.ShippedDate ) as 'DaysToShip'
	  ,DATEDIFF ( day , sO.OrderDate , sO.RequireDate ) as 'DaysTilRequired'
	  , 1 as 'OrderCount'
FROM NORHTWIND.dbo.Orders sO
	INNER JOIN TargetMart.dim.Calendar tC
	ON cast(so.OrderDate as DATE) = tc.DateValue
	INNER JOIN TargetMart.dim.Customers tCus
	ON sO.CustomerID = tCus.CustomerID
; 
End

(830 rows affected)

Total execution time: 00:00:00.043

### **_Simplicity :  Inserts data into F.OrderPerf  <span style="color: var(--vscode-foreground);">by selecting relevant information from the 'NORTHWIND.dbo.Orders' source table. The script joins with 'TargetMart.dim.Calendar' and 'TargetMart.dim.Customers' to link order details with corresponding calendar dates and customer information. Calculated columns, such as 'DaysToShip' and 'DaysTilRequired,' provide additional insights. Each inserted record is marked with 'OrderCount' equal to 1.</span>_**

In [7]:
Select top 10 * from f.OrderPerf;

(10 rows affected)

Total execution time: 00:00:00.009

OrderID,fkCalendar,fkCustomer,fkShipper,fkEmployee,Freight,DaysToShip,DaysTilRequired,OrderCount
10248,120551,1085,3,5,32.38,12,28,1
10249,120552,1079,1,6,11.61,5,42,1
10250,120555,1033,2,4,65.83,4,28,1
10251,120555,1084,1,3,41.34,7,28,1
10252,120556,1076,2,4,51.30,2,28,1
10253,120557,1033,2,3,58.17,6,14,1
10254,120558,1013,2,5,22.98,12,28,1
10255,120559,1068,3,9,148.33,3,28,1
10256,120562,1088,2,3,13.97,2,28,1
10257,120563,1034,3,4,81.91,6,28,1


## **_Product Perf Fact table_**

In [147]:
BEGIN
TRUNCATE TABLE TargetMart2.f.ProductPerf

INSERT INTO TargetMart2.f.ProductPerf(OrderID, OrderDate, ProductID, CustomerID, UnitPrice, Quantity, Discount, DiscFlag, LineTotal)
SELECT sDet.OrderID
      ,tCal.pkCalendar as 'OrderDate'
      ,sDet.ProductID + 10000 as 'ProductID'
	  ,tCust.pkCustomer
      ,sDet.UnitPrice
      ,sDet.Quantity
      ,sDet.Discount
	  ,CASE 
		 WHEN sDet.Discount = 0 THEN 0
		 ELSE 1
	   END as 'DiscFlag'
	  ,CAST(((1 - sDet.Discount) * sDet.UnitPrice) * sDet.Quantity as MONEY) as 'LineTotal'
FROM NORHTWIND.dbo.OrderDetails sDet
	INNER JOIN NORHTWIND.dbo.Orders sOrd
	ON sDet.OrderID = sOrd.OrderID
	INNER JOIN dim.Calendar tCal
	ON CAST(sOrd.OrderDate as DATE) = tCal.DateValue
	INNER JOIN dim.Customers tCust
	ON sOrd.CustomerID = tCust.CustomerID
;

END

(2155 rows affected)

Total execution time: 00:00:00.072

**_Simplicity: This is_** **_TargetMart2.f.ProductPerf' table and inserts data by selecting details from 'NORTHWIND.dbo.OrderDetails.' The script joins with 'NORTHWIND.dbo.Orders,' 'dim.Calendar,' and 'dim.Customers' to link order details with corresponding calendar dates and customer information. Calculated columns, such as 'DiscFlag' and 'LineTotal,' provide additional insights into product performance_**

In [8]:
Select top 10 * from f.ProductPerf;

(10 rows affected)

Total execution time: 00:00:00.016

OrderID,OrderDate,ProductID,CustomerID,UnitPrice,Quantity,Discount,DiscFlag,LineTotal
10248,120551,10011,1085,14.00,12,0,0,168.00
10248,120551,10042,1085,9.80,10,0,0,98.00
10248,120551,10072,1085,34.80,5,0,0,174.00
10249,120552,10014,1079,18.60,9,0,0,167.40
10249,120552,10051,1079,42.40,40,0,0,1696.00
10250,120555,10041,1033,7.70,10,0,0,77.00
10250,120555,10051,1033,42.40,35,0.15000000596046448,1,1261.40
10250,120555,10065,1033,16.80,15,0.15000000596046448,1,214.20
10251,120555,10022,1084,16.80,6,0.05000000074505806,1,95.76
10251,120555,10057,1084,15.60,15,0.05000000074505806,1,222.30


**Thanks & Happy Learning........**